In [ ]:
import pandas
import numpy
import matplotlib.pyplot
import nose.tools
%matplotlib inline
# Data files:
device = pandas.read_csv('device_info.csv')
email = pandas.read_csv('email_info.csv')
employee = pandas.read_csv('employee_info.csv')
http = pandas.read_csv('http_info.csv')
logon = pandas.read_csv('logon_info.csv')